In [1]:
import torch, numpy as np, random
seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)


In [2]:
import scanpy as sc
import numpy as np

ST = sc.read_h5ad('/data/keerthana/integration/scMODAL/xenium_breast_cancer.h5ad')
ST.var_names_make_unique()
SC = sc.read_h5ad('/data/keerthana/integration/scMODAL/xenium_sc_data.h5ad')
SC.var_names_make_unique()

/home/keerthana/anaconda3/envs/scmodal_scviva/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [3]:
ST.shape

(167780, 313)

In [4]:
SC.shape

(100064, 29733)

In [4]:
np.unique(ST.obs['cell_type'])

array(['B_Cells', 'CD4+_T_Cells', 'CD8+_T_Cells', 'DCIS_1', 'DCIS_2',
       'Endothelial', 'IRF7+_DCs', 'Invasive_Tumor', 'LAMP3+_DCs',
       'Macrophages_1', 'Macrophages_2', 'Mast_Cells', 'Myoepi_ACTA2+',
       'Myoepi_KRT15+', 'Perivascular-Like', 'Prolif_Invasive_Tumor',
       'Stromal', 'Stromal_&_T_Cell_Hybrid', 'T_Cell_&_Tumor_Hybrid',
       'Unlabeled'], dtype=object)

In [5]:
import pandas as pd

# Define mapping from fine-grained ST types to major SC types
mapping = {
    # Lymphoid
    'B_Cells': 'B-cells',
    'CD4+_T_Cells': 'T-cells',
    'CD8+_T_Cells': 'T-cells',
    'Stromal_&_T_Cell_Hybrid': 'T-cells',
    'T_Cell_&_Tumor_Hybrid': 'T-cells',

    # Myeloid
    'Macrophages_1': 'Myeloid',
    'Macrophages_2': 'Myeloid',
    'Mast_Cells': 'Myeloid',
    'IRF7+_DCs': 'Myeloid',
    'LAMP3+_DCs': 'Myeloid',
    'TFR+_DCs': 'Myeloid',

    # Epithelial
    'Invasive_Tumor': 'Cancer Epithelial',
    'Prolif_Invasive_Tumor': 'Cancer Epithelial',
    'DCIS_1': 'Cancer Epithelial',
    'DCIS_2': 'Cancer Epithelial',
    'Myoepi_ACTA2+': 'Normal Epithelial',
    'Myoepi_KRT15+': 'Normal Epithelial',

    # Endothelial
    'Endothelial': 'Endothelial',
    'Perivascular-Like': 'Endothelial',

    # Stromal
    'Stromal': 'CAFs',

    # Unlabeled / others
    'Unlabeled': 'Unlabeled'
}

# Apply mapping
ST.obs['celltype_major'] = ST.obs['cell_type'].map(mapping)

# If any fine types weren't mapped
unmapped = ST.obs.loc[ST.obs['celltype_major'].isna(), 'cell_type'].unique()
if len(unmapped) > 0:
    print("Unmapped cell types:", unmapped)
else:
    print("✅ All cell types mapped successfully!")



✅ All cell types mapped successfully!


In [6]:
# Reorder ST and SC so that common columns are first and in the same order
common_vars = ST.var_names.intersection(SC.var_names)
other_ST = [v for v in ST.var_names if v not in common_vars]
other_SC = [v for v in SC.var_names if v not in common_vars]
ST = ST[:, list(common_vars) + other_ST]
SC = SC[:, list(common_vars) + other_SC]



In [7]:
import scmodal

In [8]:
model = scmodal.model.Model(training_steps=10000, device = 'gpu')

In [9]:
model.preprocess(ST,SC, shared_gene_num=len(common_vars),alpha = None, eta = None)

preprocess called with alpha: None and eta: None
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [10]:
import numpy as np
model.train()

Begining time:  Thu Nov  6 22:02:00 2025
step 0, loss_D=0.653453, loss_GAN=-0.273153, loss_AE=813.897156, loss_Geo=-7.278175, loss_LA=185312.343750, loss_MNN=124.283852
step 2000, loss_D=0.586172, loss_GAN=-0.548363, loss_AE=6.432248, loss_Geo=-18.877806, loss_LA=0.395344, loss_MNN=1.018513
step 4000, loss_D=0.677326, loss_GAN=-0.656650, loss_AE=6.553828, loss_Geo=-18.882704, loss_LA=0.404291, loss_MNN=1.174940
step 6000, loss_D=0.805292, loss_GAN=-0.783314, loss_AE=7.612850, loss_Geo=-18.786213, loss_LA=0.610302, loss_MNN=1.730116
step 8000, loss_D=0.982300, loss_GAN=-0.967140, loss_AE=5.768522, loss_Geo=-18.918793, loss_LA=0.329216, loss_MNN=1.098849
Ending time:  Fri Nov  7 01:32:56 2025
Training takes 12655.89 seconds


In [11]:
model.eval()

Begining time:  Fri Nov  7 01:32:57 2025
Ending time:  Fri Nov  7 01:33:15 2025
Evaluating takes 17.80 seconds


In [12]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd

def impute_cell_types(model, ST, SC, n_neighbors=10):
    emb_A = model.latent[:ST.shape[0], :]
    emb_B = model.latent[ST.shape[0]:, :]

    nbrs = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine').fit(emb_B)
    knn_idx = nbrs.kneighbors(emb_A, return_distance=False)

    # Factorize string labels to numeric
    labels = np.array(SC.obs["celltype_major"].values)
    codes, uniques = pd.factorize(labels)
    neighbor_codes = codes[knn_idx]

    # Majority vote using bincount (fast + numeric)
    majority_idx = np.apply_along_axis(lambda x: np.bincount(x).argmax(), 1, neighbor_codes)
    majority_labels = uniques[majority_idx]

    ST.obs["imputed_cell_type"] = majority_labels
    return ST


In [13]:
ST = impute_cell_types(model, ST, SC, n_neighbors=10)

/tmp/ipykernel_2029848/3903814323.py:21: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ST.obs["imputed_cell_type"] = majority_labels


In [14]:
#compute ari between imputed cell types and actual cell types
from sklearn.metrics import adjusted_rand_score
ari = adjusted_rand_score(ST.obs['celltype_major'], ST.obs['imputed_cell_type'])
ari

0.5161987703319615

In [15]:
from sklearn.metrics import adjusted_rand_score
model = scmodal.model.Model(training_steps=100, device = 'gpu')
model.preprocess(ST,SC, shared_gene_num=len(common_vars),alpha = None, eta = None)

preprocess called with alpha: None and eta: None
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [16]:
n_iter = 10
aris = []
for i in range(n_iter):
    print(f"🔄 Iteration {i+1}/{n_iter}")
    if i == 0:
        model.train(load_ckpt=False)  # Start from scratch
    else:
        model.train(load_ckpt=True)  # Continue from saved weights
    model.eval()

    ST = impute_cell_types(model, ST, SC, n_neighbors=10)

    #keep only first word

    ari = adjusted_rand_score(ST.obs['celltype_major'], ST.obs['imputed_cell_type'])
    aris.append(ari)
    print(f"Iteration ARI: {ari}")

    # if i > 0 and abs(ari - prev_ari) < 0.005:
    #     print("Converged ✅")
    #     break
    prev_ari = ari


🔄 Iteration 1/10
Begining time:  Fri Nov  7 01:38:51 2025
step 0, loss_D=0.653453, loss_GAN=-0.273153, loss_AE=813.897156, loss_Geo=-7.278175, loss_LA=185312.343750, loss_MNN=124.283852
Ending time:  Fri Nov  7 01:40:33 2025
Training takes 101.31 seconds
Begining time:  Fri Nov  7 01:40:34 2025
Ending time:  Fri Nov  7 01:40:40 2025
Evaluating takes 5.58 seconds
Iteration ARI: 0.0011854073537871939
🔄 Iteration 2/10
Begining time:  Fri Nov  7 01:46:06 2025
🧩 Loading checkpoint with shape-safe mapping...
⚠️ Skipping optimizer_G load due to mismatch: local variable 'optimizer_G' referenced before assignment
⚠️ Skipping optimizer_D load due to mismatch: local variable 'optimizer_D' referenced before assignment
step 0, loss_D=0.182114, loss_GAN=-0.164531, loss_AE=26.203583, loss_Geo=-18.338282, loss_LA=15.943383, loss_MNN=1.554775
Ending time:  Fri Nov  7 01:47:51 2025
Training takes 104.91 seconds
Begining time:  Fri Nov  7 01:47:53 2025
Ending time:  Fri Nov  7 01:47:58 2025
Evaluating ta

In [17]:
aris

[0.0011854073537871939,
 0.4665392011349993,
 0.6067394078817228,
 0.3142171210390439,
 0.29798062976897166,
 0.4327936357444938,
 0.4389937108198339,
 0.517007295049662,
 0.43124429711203116,
 0.47707345741740037]

In [18]:
#compute pearson correlation between